In [177]:
import numpy as np
import pickle
from elements import *
from tqdm import tqdm
from ai_strategies import *
import random

In [198]:
import util

In [263]:
white = 'w'
black = 'b'

class Q_Game():
    def __init__(self, board=None, turn=None):
        if board:
            self.board = board
        else:
            self.board = Board()
        if turn:
            self.history = [turn]
        else:
            self.history = []
            
    def initiate_game(self, r0=None):
        if r0:
            pass
        else:
            r0 = Roll()
        if r0.roll[0]>r0.roll[1]:
            return Turn(white, r0)
        elif r0.roll[1]>r0.roll[0]:
            return Turn(black, r0)
        else:
#             self.board.value *= 2
            return self.initiate_game()

    @classmethod
    def computer_interact(cls, board, turn, strat, eps=1, simple=True, _print=True):
        scores = {}
        if _print:
            print '{} Computer rolled: '.format(turn.player.upper()), turn.roll
        col = turn.player
        enemy = white if col == black else black
        # print board.possible_moves(turn)
        if not all(m==[] for m in board.possible_moves(turn).values()):
            if len(turn.roll.touse) == 2:
                for d, ms in board.possible_moves(turn).items():
                    for m in ms:
                        temp_turn = turn.copy()
                        temp_turn.roll.use(d)
                        temp_board = board.move(col, *m)
                        if temp_board.is_complete()[0]:
                            if _print:
                                print '{} Computer played: '.format(col), m
                            return temp_board, Turn(enemy, Roll())
                        poss = temp_board.possible_moves(temp_turn)
                        if all(len(v)>0 for v in poss.values()):
                            for dd, mms in poss.items():
                                for mm in mms:
                                    if (tuple(m), tuple(mm)) in scores.keys():
                                        continue
                                    temp_final = temp_board.move(col, *mm)
                                    if simple:
                                        scores[(tuple(m), tuple(mm))] = ai_strategies.score(strat, temp_final.get_features(board, col))
                                    else:
                                        scores[(tuple(m), tuple(mm))] = ai_strategies.score(strat, temp_final.feat(board, col))
                        else:
                            if simple:
                                scores[(tuple(m), ' ')] = ai_strategies.score(strat, temp_board.get_features(board, col))
                            else:
                                scores[(tuple(m), ' ')] = ai_strategies.score(strat, temp_board.feat(board, col))
            else:
                poss = board.possible_moves(turn)
                if len(poss)>0 and any(len(x)>0 for x in poss.values()):
                    for m in poss.values()[0]:
                        t = turn.copy()
                        t.roll.use(t.roll.touse[0])
                        if board.move(col, *m).is_complete()[0]:
                            if _print:
                                print '{} Computer played: '.format(col), m
                            return board.move(col, *m), Turn(enemy, Roll())
                        poss1 = board.move(col, *m).possible_moves(t)
                        if len(poss1)>0 and any(len(x)>0 for x in poss1.values()):
                            for m2 in poss1.values()[0]:
                                t1 = turn.copy()
                                t1.roll.use(t1.roll.touse[0])
                                if board.move(col, *m).move(col, *m2).is_complete()[0]:
                                    if _print:
                                        print '{} Computer played: '.format(col), m, m2
                                    return board.move(col, *m).move(col, *m2), Turn(enemy, Roll())
                                poss2 = board.move(col, *m).move(col, *m2).possible_moves(t1)
                                if len(poss2)>0 and any(len(x)>0 for x in poss2.values()):
                                    for m3 in poss2.values()[0]:
                                        t2 = turn.copy()
                                        t2.roll.use(t2.roll.touse[0])
                                        if board.move(col, *m).move(col, *m2).move(col, *m3).is_complete()[0]:
                                            if _print:
                                                print '{} Computer played: '.format(col), m, m2, m3
                                            return board.move(col, *m).move(col, *m2).move(col, *m3), Turn(enemy, Roll())
                                        poss3 = board.move(col, *m).move(col, *m2).move(col, *m3).possible_moves(t2)
                                        if len(poss3)>0 and any(len(x)>0 for x in poss3.values()):
                                            for m4 in poss3.values()[0]:
                                                if board.move(col, *m).move(col, *m2).move(col, *m3).move(col, *m4).is_complete()[0]:
                                                    if _print:
                                                        print '{} Computer played: '.format(col), m, m2, m3, m4
                                                    return board.move(col, *m).move(col, *m2).move(col, *m3).move(col, *m4), Turn(enemy, Roll())
                                                temp_board = board.move(col, *m).move(col, *m2).move(col, *m3).move(col, *m4)
                                                k = (tuple(m), tuple(m2), tuple(m3), tuple(m4))
                                                if simple:
                                                    scores[k] = ai_strategies.score(strat, temp_board.get_features(board, col))
                                                else:
                                                    scores[k] = ai_strategies.score(strat, temp_board.feat(board, col))
                                        else:
                                            temp_board = board.move(col, *m).move(col, *m2).move(col, *m3)
                                            k = (tuple(m), tuple(m2), tuple(m3))
                                            if simple:
                                                scores[k] = ai_strategies.score(strat, temp_board.get_features(board, col))
                                            else:
                                                scores[k] = ai_strategies.score(strat, temp_board.feat(board, col))
                                else:
                                    temp_board = board.move(col, *m).move(col, *m2)
                                    k = (tuple(m), tuple(m2))
                                    if simple:
                                        scores[k] = ai_strategies.score(strat, temp_board.get_features(board, col))
                                    else:
                                        scores[k] = ai_strategies.score(strat, temp_board.feat(board, col))

                        else:
                            temp_board = board.move(col, *m)
                            k = (tuple(m), ' ')
                            if simple:
                                scores[k] = ai_strategies.score(strat, temp_board.get_features(board, col))
                            else:
                                scores[k] = ai_strategies.score(strat, temp_board.feat(board, col))
            all_ = scores.items()
            values = [x[1] for x in all_]
            maxIndex = values.index(max(values))
            if random.uniform(0,1)<eps:
                best_moves = all_[maxIndex][0]
            else:
                best_moves = all_[random.randint(0, len(all_)-1)][0]
            for m in best_moves:
                if m == ' ':
                    continue
                else:
                    board = board.move(col, *m)
            if _print:
                print '{} Computer played: '.format(col.upper()), best_moves
        else:
            if _print:
                print '{} Computer cannot move, your turn !'.format(col)
        next_t = Turn(enemy, Roll())
        while next_t.roll.roll in [[1, 1], [2,2], [3,3], [4,4], [5,5], [6,6]]:
            next_t = Turn(enemy, Roll())
        return  board, next_t
    
    def update(self, comp_strat, og_board, sa, sprime, turn, r, gamma, alpha):
        possible_actions = sprime.get_pairs_moves(turn)
        Qs = []
        f = sa.feat(og_board, turn.player)
        if r == 0:
            if len(possible_actions) == 0:
                return comp_strat
            for action in possible_actions:
                sprime2 = sprime.copy()
                for m in action:
                    if m == ' ':
                        continue
                    else:
                        sprime2 = sprime2.move(turn.player, *m)
                Qs.append(score(comp_strat, sprime2.feat(sprime, turn.player)))
            diff = gamma*max(Qs)-score(comp_strat, f)
        else:
            diff = r - score(comp_strat, sa.feat(og_board, turn.player))
        new_w = []
#         print diff
        for i in range(len(comp_strat.asList)):
            new_w.append(comp_strat.asList[i]+alpha*diff*f[i])
#         for i in range(len(comp_strat.asList)):
#             new_w[i] *= 1./max(new_w)
        return SimpleWeights(new_w, comp_strat.type, comp_strat.name)
    
    def qlearning(self, comp_strat, adv, gamma, alpha, _print=True):
        toimprove = SimpleWeights(comp_strat.asList, True, 'toimprove')
        turn = self.initiate_game(Roll())
#         print turn.player
        other = white if turn.player == black else black
        strat = {}
        strat[turn.player] = comp_strat 
        strat[other] = comp_strat
        while True:
            r=0
            shouldBreak = False
#             print turn
            sa, turn = self.computer_interact(self.board, turn, toimprove, 0.8, False, _print=_print)
#             print sa
#             print '****************************************************'
            if sa.is_complete()[0] and sa.is_complete()[1][0].lower() != other:
#                 if sa.is_complete()[2]:
#                     r = 10
#                 else:
#                     r = 5
                r=20
                shouldBreak = True
            else:
#                 print turn
                sprime, turn = self.computer_interact(sa, turn, adv, 1, False, _print=_print)
#                 print sprime
#                 print '****************************************************'
                if sa.is_complete()[0] and sa.is_complete()[1][0].lower() == other:
#                     if sa.is_complete()[2]:
#                         r = 5
#                     else:
#                         r = 10
                    r=-5
                    shouldBreak = True
            toimprove = self.update(toimprove, self.board, sa, sprime, turn, r, gamma, alpha)
            if shouldBreak:
                break
            else:
                self.board = sprime
#         print r
        return toimprove

def run_qlearning(strat_to_improve, adv, gamma, alpha, max_iteration=100):
    if isinstance(strat_to_improve, list):
        champs = strat_to_improve
    else:
        champs=[strat_to_improve]
    for i in range(max_iteration):
        print "{}/{}".format(i+1, max_iteration)
        g = Q_Game()
        champs.append(g.qlearning(champs[-1], adv, gamma, alpha, _print=False))
    return champs

In [251]:
qlog = logistic_long.copy()
qlog.name = 'champion'
qlog2 = logistic_long.copy()
st = SimpleWeights(numpy.random.rand(41), True, 'rand')

In [237]:
%%time 
qlog=Q_Game().qlearning(qlog, qlog2, 0.1, 0.00001, _print=False)

CPU times: user 6.21 s, sys: 60.8 ms, total: 6.27 s
Wall time: 6.33 s


In [267]:
%%time
new2 = run_qlearning(new2, qlog2, 0.5, 0.000001, max_iteration=100)

1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100
CPU times: user 13min 10s, sys: 8.5 s, total: 13min 19s
Wall time: 13min 49s


In [212]:
%%time
new = run_qlearning(new, qlog2, 0.8, 0.000001, max_iteration=1000)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [268]:
for v in zip(new2[-1].asList, qlog2.asList):
    print v

(10.513928453727098, 10.318737605516487)
(0.03665897636716714, 0.0013386402458066844)
(-10.383521078312361, -28.763395470699223)
(-1.9812681320846859, -3.2693470834189182)
(-7.2268173161494094, -7.5160214978927877)
(-7.1934974631247792, -10.090222951710532)
(-0.058559108604641211, -0.10626398492137497)
(0.14603343061814536, -1.2115813452055832)
(-0.15687361404182876, -1.5579815695755994)
(-1.5940895247508318, -1.8518047722292628)
(-1.1278465977883321, -1.5632234460084748)
(-0.1358589500191868, -0.2849955722648515)
(0.46214445743694715, -0.23344958407171906)
(1.2156356424217336, -1.5079036413128504)
(-0.55597493779631557, -0.97123902761147085)
(-1.3658852329825804, -1.4937315684080901)
(-0.12275466238407588, -0.24734196960562541)
(0.10202476751770939, -0.034877831980125319)
(-1.4574258604994315, -2.8199529196085433)
(-0.23711859754993708, -1.1220755757898206)
(1.0878489704323944, -0.24827917589908738)
(-0.037370307502920044, -0.037370307502920044)
(-40.93975833543071, -53.76012494960627

In [269]:
len(new2)

201

In [272]:
util.play_n(new2[-1], qlog2, 10, show=True)

Champ: 0
Chal: 11


(0, 11)

In [240]:
with open('data/champs_qlearning.pkl', 'w') as f:
    pickle.dump(new, f)
with open('data/champs_qlearning1.pkl', 'w') as f:
    pickle.dump(new1, f)

In [273]:
with open('data/testing.pkl', 'r') as f:
    test_data = pickle.load(f)

def test(plays, w):
    pos = 0 
    c = 0
    for p in tqdm(plays):
        board, t, poss_moves, cl, moves = p
        if len(poss_moves)<=1:
            continue
        c+=1
        probs = []
        for j, ms in enumerate(poss_moves):
            temp = board.copy()
            for m in ms:
                temp = temp.move(t.player, *m)
            f = temp.feat(board, t.player)
            if j == cl:
                f_j = f
            probs.append(score(w, f))
        probs = softmax(probs)
        if probs[cl] == max(probs): 
            pos+=1
    return float(pos)/c

def softmax(s):
    num = [np.exp(s[i])+0.00001 for i in range(len(s))]
    return [x/sum(num) for x in num]

In [275]:
test(test_data, new2[-1])

100%|██████████| 2677/2677 [01:04<00:00, 41.54it/s]


0.8603002502085071

In [276]:
 new2[-1]

toimprove: [10.513928453727098, 0.03665897636716714, -10.383521078312361, -1.9812681320846859, -7.2268173161494094, -7.1934974631247792, -0.058559108604641211, 0.14603343061814536, -0.15687361404182876, -1.5940895247508318, -1.1278465977883321, -0.1358589500191868, 0.46214445743694715, 1.2156356424217336, -0.55597493779631557, -1.3658852329825804, -0.12275466238407588, 0.10202476751770939, -1.4574258604994315, -0.23711859754993708, 1.0878489704323944, -0.037370307502920044, -40.93975833543071, -3.3731213853774831, -17.443373294193606, -11.285669866742879, -0.059228216559662754, -4.6652175406332015, -3.663706250577957, -3.7849705874744868, -1.5357370825787495, 0.041129684329487506, -1.8851695285103183, -1.2146105759529324, -1.634758410668683, -2.169318540721386, -0.027724832764959715, -0.55917520234064622, -2.8149248399111966, -2.4056688743066532, 0.40174027284356889]